In [8]:
#Caitlin Mariel Lindsay 
#Discrete Structures III Semi-Final Exam

In [41]:
#Data and Preprocessing
#A lot from Andrea Baulita

def load_arff(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    # Find the start of data
    data_start = lines.index('@data\n') + 1

    # Get attribute names
    attributes = [line.split()[1] for line in lines if line.startswith('@attribute')]

    # Initialize an empty list to store dictionaries
    data_list = []

    # Parse the data and convert it into dictionaries
    for line in lines[data_start:]:
        values = line.strip().split(',')
        data_dict = {attr: val if val != 'm' else None for attr, val in zip(attributes, values)}
        data_list.append(data_dict)

    return data_list


# Function for additional preprocessing (replace 'm' with None)
def preprocess_data(data_list):
    for entry in data_list:
        for key, value in entry.items():
            if value == 'm':
                entry[key] = None

    return data_list

def linear_interpolation(data_list):
    for i in range(1, len(data_list) - 1):
        for key, value in data_list[i].items():
            if value is None and data_list[i - 1][key] is not None and data_list[i + 1][key] is not None:
                # Convert string to numeric type before performing division
                data_list[i][key] = (float(data_list[i - 1][key]) + float(data_list[i + 1][key])) / 2

    return data_list

def z_score_standardization(matrix):
    for i in range(2, len(matrix[0])):
        # Extract non-None and non-'m' values
        column = [float(row[i]) for row in matrix if row[i] is not None and row[i] != 'm']

        # Check for zero variance
        if len(set(column)) == 1:
            # Skip columns with zero variance
            continue

        mean_val = sum(column) / len(column)

        # Check for zero standard deviation
        if len(column) > 1:
            std_dev = (sum((x - mean_val) ** 2 for x in column) / len(column)) ** 0.5
        else:
            std_dev = 0

        for row in matrix:
            if row[i] is not None and row[i] != 'm':
                # Check for NaN values
                if std_dev != 0:
                    row[i] = (float(row[i]) - mean_val) / std_dev
                else:
                    row[i] = 0  # Assigning a default value for zero standard deviation

    return matrix

def impute_data(matrix):
    """
    Impute missing values in a matrix using mean imputation.

    Parameters:
    - matrix (list of lists): The input matrix with missing values represented as 'None'.

    Returns:
    - imputed_matrix (list of lists): The matrix with missing values imputed.
    """
    # Transpose the matrix for easier column-wise operations
    transposed_matrix = list(map(list, zip(*matrix)))

    # Iterate over each column
    for col_idx, column in enumerate(transposed_matrix):
        # Extract non-None values for mean calculation
        values = [float(val) for val in column if val is not None and isinstance(val, (int, float, str))]

        # Calculate the mean of non-None values
        mean_value = sum(values) / len(values) if values else None

        # Replace None values with the mean
        transposed_matrix[col_idx] = [mean_value if val is None else float(val) for val in column]

    # Transpose the matrix back to its original shape
    imputed_matrix = list(map(list, zip(*transposed_matrix)))

    return imputed_matrix



def display_data_table(data_list):
    # Get the attribute names
    attributes = list(data_list[0].keys())

    # Calculate the maximum width for each column
    column_widths = {attr: max(len(attr), max(len(str(entry[attr])) for entry in data_list)) for attr in attributes}

    # Print header
    header = "|".join(f"{attr:^{column_widths[attr]}}" for attr in attributes)
    print(header)
    print("-" * sum(column_widths.values()))

    # Print data rows
    for entry in data_list:
        row = "|".join(f"{str(entry[attr]):^{column_widths[attr]}}" if entry[attr] is not None else 'm' for attr in attributes)
        print(row)

def display_matrix(matrix):
    for row in matrix:
        print("|".join(f"{str(cell):^10}" for cell in row))


def get_matrix_dimensions(matrix):
    if not matrix or not matrix[0]:
        return 0, 0  # Matrix is empty

    num_rows = len(matrix)
    num_cols = len(matrix[0])

    return num_rows, num_cols

def remove_duplicates(matrix):
    # Convert each row to a tuple for easy comparison
    unique_rows = {tuple(row) for row in matrix}

    # Convert back to a list of lists
    deduplicated_matrix = [list(row) for row in unique_rows]

    return deduplicated_matrix

def crop_matrix(matrix, start_row, end_row, start_col, end_col):
    if not matrix or not matrix[0]:
        return []  # Empty matrix

    # Ensure indices are within bounds
    start_row = max(0, start_row)
    end_row = min(len(matrix), end_row)
    start_col = max(0, start_col)
    end_col = min(len(matrix[0]), end_col)

    cropped_matrix = [row[start_col:end_col] for row in matrix[start_row:end_row]]

    return cropped_matrix

# locally loading and preprocessing data for each year
file_path_2017 = r'C:\Users\Cait\Documents\Documents\School 2023-2024\First Sem\CS 3101N -DS3\SF\visegrad+group+companies+data-2\V4 data\2017.arff'
file_path_2018 = r'C:\Users\Cait\Documents\Documents\School 2023-2024\First Sem\CS 3101N -DS3\SF\visegrad+group+companies+data-2\V4 data\2018.arff'
file_path_2019 = r'C:\Users\Cait\Documents\Documents\School 2023-2024\First Sem\CS 3101N -DS3\SF\visegrad+group+companies+data-2\V4 data\2019.arff'
file_path_2020 = r'C:\Users\Cait\Documents\Documents\School 2023-2024\First Sem\CS 3101N -DS3\SF\visegrad+group+companies+data-2\V4 data\2020 Q4.arff'
file_path_2021 = r'C:\Users\Cait\Documents\Documents\School 2023-2024\First Sem\CS 3101N -DS3\SF\visegrad+group+companies+data-2\V4 data\2021 Q1.arff'

# # to make loading data compatible with github
# import os

data_2017 = load_arff(file_path_2017)
data_2018 = load_arff(file_path_2018)
data_2019 = load_arff(file_path_2019)
data_2020 = load_arff(file_path_2020)
data_2021 = load_arff(file_path_2021)

data_2017_preprocessed = preprocess_data(data_2017)
data_2018_preprocessed = preprocess_data(data_2018)
data_2019_preprocessed = preprocess_data(data_2019)
data_2020_preprocessed = preprocess_data(data_2020)
data_2021_preprocessed = preprocess_data(data_2021)

# perform linear interpolation for each year
data_2017_preprocessed = linear_interpolation(data_2017_preprocessed)
data_2018_preprocessed = linear_interpolation(data_2018_preprocessed)
data_2019_preprocessed = linear_interpolation(data_2019_preprocessed)
data_2020_preprocessed = linear_interpolation(data_2020_preprocessed)
data_2021_preprocessed = linear_interpolation(data_2021_preprocessed)

data_2017_preprocessed = preprocess_data(data_2017)
data_2018_preprocessed = preprocess_data(data_2018)
data_2019_preprocessed = preprocess_data(data_2019)
data_2020_preprocessed = preprocess_data(data_2020)
data_2021_preprocessed = preprocess_data(data_2021)

data_2017_preprocessed = linear_interpolation(data_2017_preprocessed)
data_2018_preprocessed = linear_interpolation(data_2018_preprocessed)
data_2019_preprocessed = linear_interpolation(data_2019_preprocessed)
data_2020_preprocessed = linear_interpolation(data_2020_preprocessed)
data_2021_preprocessed = linear_interpolation(data_2021_preprocessed)


data_combined = data_2017_preprocessed + data_2018_preprocessed + data_2019_preprocessed + data_2020_preprocessed + data_2021_preprocessed



attributes = list(data_combined[0].keys())
#display_data_table(data_combined)


matrix = []
for entry in data_combined:
    row = [entry[attr] for attr in attributes[2:]] 
    matrix.append(row)

standardized_data = z_score_standardization(matrix)
FilledMatrix = impute_data(standardized_data)
#deduplicated_data = remove_duplicates(FilledMatrix)
croppedMatrix = crop_matrix(FilledMatrix, 0, 10, 0, 10)

#num_rows, num_cols = get_matrix_dimensions(FilledMatrix)
#num_rows2, num_cols2 = get_matrix_dimensions(deduplicated_data)

print("Number of Rows:", num_rows)
print("Number of Columns:", num_cols)

#print("Number of Rows2:", num_rows2)
#print("Number of Columns2:", num_cols2)
#display_matrix(FilledMatrix)
display_matrix(croppedMatrix)







Number of Rows: 2250
Number of Columns: 83
   0.14   |   0.53   |0.0032818118460023454|-0.05933481047303196|-0.02090743833054709|0.022837062860168126|-0.06619743358084555|-0.021484331809106232|-0.021478625671629312|0.022845940141746037
   0.01   |   0.5    |0.0004994166177466374|-0.05961591915645378|-0.021005159145065972|0.02237460396819302|-0.06400005757850212|-0.021487049787946598|-0.021478346712524108|0.022215293187791505
   0.03   |   0.74   |-0.0008917809963812167|-0.07029804912648309|-0.021026874881625725|0.02237460396819302|-0.07608562559139102|-0.021485279941259847|-0.021480578385365733|0.02229937944831878
   0.0    |   0.58   |0.002354346769917109|-0.06270811467409385|-0.020947250514239967|0.022290520533288453|-0.06931038291749876|-0.021485785611741776|-0.021479090603471314|0.022173250057527873
   0.0    |   0.0    |-0.0011236472654025257|-0.09897113483550915|-0.021026874881625725|0.02224847881583617|-0.08249463893155937|-0.021491158360612272|-0.021482996030944165|0.0221312069

In [40]:
#SVD
#create a function to initialize matrix
def initmatrix():
    givenData = croppedMatrix
    
    #for row in givenData:
        #print(row)
        
    AT = transpose(givenData)
    multiplyU(givenData,AT)
    multiplyV(AT,givenData)
    
#create a function that transposes
def transpose(matrix):
    rows = len(matrix)
    cols = len(matrix[0])
    
    newMatrix = [[0 for _ in range(rows)] for _ in range(cols)]
    
    for x in range(rows): #row
        for y in range (cols): #col
            newMatrix[y][x] = matrix[x][y]
            
    #print('\n')       
    #for row in newMatrix:
        #print(row)
    return newMatrix     
    
        
        
#create a function that multiplies matrices
def multiplyU(matrix, transposed):
    rows1, cols1 = len(matrix), len(matrix[0])
    rows2, cols2 = len(transposed), len(transposed[0])

    # Check if matrices can be multiplied
    if cols1 != rows2:
        raise ValueError("Matrices cannot be multiplied. Number of columns in the first matrix must be equal to the number of rows in the second matrix.")

    # Initialize the result matrix with zeros
    result = [[0 for _ in range(cols2)] for _ in range(rows1)]

    #matrix multiplication
    for i in range(rows1):
        for j in range(cols2):
            for k in range(cols1):  # or rows2 since they are equal
                result[i][j] += matrix[i][k] * transposed[k][j]
    #print("Matrix:")
    #for mat_row, transposed_row, result_row in zip(matrix, transposed, result):
        #print(f"{mat_row} * {transposed_row} = {result_row}")
        
    #print('\n')       
    #for row in result:
        #print(row)
    #print(result)
    
    #det = determinant(result)
    #print("\ndet is", det)
    e = eigen(result,rows1)
    #print("\nEigenvalues:", e) 
    U = transpose(e[1])
    print("U")
    for row in U:
        print(row)
    S = sigma(e[0],matrix)
    print("Sigma")
    for row in S:
        print(row)

def multiplyV(transposed, matrix):
    rows1, cols1 = len(transposed), len(transposed[0])
    rows2, cols2 = len(matrix), len(matrix[0])

    # Check if matrices can be multiplied
    if cols1 != rows2:
        raise ValueError("Matrices cannot be multiplied. Number of columns in the first matrix must be equal to the number of rows in the second matrix.")

    # Initialize the result matrix with zeros
    result = [[0 for _ in range(cols2)] for _ in range(rows1)]

    # matrix multiplication
    for i in range(rows1):
        for j in range(cols2):
            for k in range(cols1):  # or rows2 since they are equal
                result[i][j] += transposed[i][k] * matrix[k][j]
    #print("Matrix:")
    #for mat_row, transposed_row, result_row in zip(matrix, transposed, result):
        #print(f"{mat_row} * {transposed_row} = {result_row}")
        
    #print('\n')       
    #for row in result:
        #print(row)
    #print(result)
    
    det = determinant(result)
    #print("\ndet is", det)
    e = eigen(result,rows1)
    #print("\nEigenvalues:", e) 
    #print('\n')  
    V = e[1]
    print("V")
    for row in V:
        print(row)
    
    
    
#create a function that find determinant -> cofactor expansion
def determinant(matrix):
    """
    Recursive function to calculate the determinant of an n x n matrix using cofactor expansion.
    """
    size = len(matrix)

    if size == 1:
        return matrix[0][0]
    elif size == 2:
        return matrix[0][0] * matrix[1][1] - matrix[0][1] * matrix[1][0]

    det = 0
    for col in range(size):
        det += ((-1) ** col) * matrix[0][col] * determinant(minor(matrix, 0, col))
        #print(f"Current determinant: {det}")
        
    return det

def minor(matrix, row, col):
    """
    Calculate the minor matrix obtained by removing the specified row and column.
    """
    return [[matrix[i][j] for j in range(len(matrix[i])) if j != col] for i in range(len(matrix)) if i != row]



def eigen(matrix, num_eigenvalues):
    eigenvalues = []
    eigenvectors = []

    for _ in range(num_eigenvalues):
        eigenvalue, eigenvector, matrix = power_iteration(matrix)
        eigenvalues.append(eigenvalue)
        eigenvectors.append(eigenvector)

    return eigenvalues, eigenvectors

def initialize_random_vector(size):
    return [((i * 2654435761) % 2**32) / 2**32 for i in range(1, size + 1)]

def matrix_vector_multiply(matrix, vector):
    result = [sum(matrix[i][j] * vector[j] for j in range(len(vector))) for i in range(len(matrix))]
    return result

def dot_product(vector1, vector2):
    return sum(val1 * val2 for val1, val2 in zip(vector1, vector2))


def power_iteration(matrix, num_iterations=1000, tolerance=1e-6):
    n = len(matrix)
    #also implicitly normalizes vector
    
    # Initialize a random vector
    current_vector = initialize_random_vector(n)
    current_vector = normalize_vector(current_vector)

    for _ in range(num_iterations):
        # Apply matrix multiplication
        next_vector = matrix_vector_multiply(matrix, current_vector)

        # Normalize the vector
        next_vector = normalize_vector(next_vector)

        # Check for convergence
        if sum((next_val - current_val)**2 for next_val, current_val in zip(next_vector, current_vector)) < tolerance:
            break

        current_vector = next_vector

    # Calculate the eigenvalue
    eigenvalue = dot_product(next_vector, matrix_vector_multiply(matrix, next_vector))

    # Deflate the matrix
    matrix = matrix_deflation(matrix, eigenvalue, next_vector)

    return eigenvalue, next_vector, matrix


def matrix_deflation(matrix, eigenvalue, eigenvector):
    n = len(matrix)

    outer_product = [[eigenvalue * v1 * v2 for v1 in eigenvector] for v2 in eigenvector]

    for i in range(n):
        for j in range(n):
            matrix[i][j] -= outer_product[i][j]

    return matrix

def normalize_vector(vector):
    magnitude = (sum(val**2 for val in vector))**0.5
    return [val / magnitude for val in vector]

def makeU(eigenvectors):
    # Transpose the eigenvectors to get columns as rows
    u_matrix = [list(row) for row in zip(*eigenvectors)]
    return u_matrix

def sigma(eigenvalues, original_matrix):
    """
    Construct the Sigma matrix for Singular Value Decomposition (SVD).

    Parameters:
    - eigenvalues (list): List of eigenvalues.
    - original_matrix (list): Original matrix.

    Returns:
    - sigma_matrix (list): Rectangular matrix with singular values.
    """
    # Compute singular values from eigenvalues
    singular_values = [eigenvalue**0.5 for eigenvalue in eigenvalues]

    # Determine dimensions of the original matrix
    m, n = len(original_matrix), len(original_matrix[0])

    # Construct Sigma matrix
    sigma_matrix = [[0] * n for _ in range(m)]

    for i in range(min(m, n)):
        sigma_matrix[i][i] = singular_values[i]

    return sigma_matrix

initmatrix()


U
[0.3480254450741242, -0.1580042068278438, 0.5644379026803268, -0.6712302839050692, 0.07884897485149034, -0.2035660496031232, -0.15750768367079232, -0.00021703285682973423, -0.17157123102261795, 0.5637423165512199]
[0.3237016800833751, 0.08934927105961579, -0.09913037452766019, 0.20951771630920069, -0.06541725359507795, 0.273885585995372, 0.08925791956502722, -0.17157896386799176, -0.8352434248911778, -0.09932770177171513]
[0.47592760076351637, -0.03755572418356156, -0.14785811430957044, -0.08462153208801577, 0.43241632093032206, 0.5315965744299753, -0.037694186270989555, 0.2403211059626636, 0.18342477993565, -0.14786355285435973]
[0.37380929800916873, 0.0789135659772076, -0.20290501950416703, 0.20023466535311515, -0.025819314735428124, -0.5597803250053485, 0.07874616615112612, 0.6700089464070009, -0.07992213229342612, -0.20285930088993107]
[0.01509337133782324, 0.6345044973581255, 0.28622554052736265, 0.06485121053575073, -0.007675651779765533, 0.034796733092162656, 0.634754124650226

In [44]:
#PCA

#compute mean, variance of row and indexes. Form new matrix with computed standard deviations
def calculate_statistics_matrix_population(matrix):
    num_samples = len(matrix)
    num_features = len(matrix[0])

    # Step 1: Calculate the mean for each row
    means = [sum(matrix[i]) / num_features for i in range(num_samples)]

    # Step 2: Calculate the population variance for each element using the mean of its row
    variances = [
        [
            (matrix[i][j] - means[i])**2 / num_features for j in range(num_features)
        ]
        for i in range(num_samples)
    ]

    # Step 3: Calculate the z-score for each element using the standard deviation
    z_scores = [
        [
            (matrix[i][j] - means[i]) / (variances[i][j]**0.5) for j in range(num_features)
        ]
        for i in range(num_samples)
    ]


    return means, variances, z_scores

def create_z_scores_matrix(z_scores):
    num_samples = len(z_scores)
    num_features = len(z_scores[0])

    z_scores_matrix = [
        [z_scores[i][j] for j in range(num_features)]
        for i in range(num_samples)
    ]

    return z_scores_matrix
#compute covariance

def calculate_covariance_matrix(matrix, variances):
    num_samples = len(matrix)
    num_features = len(matrix[0])

    # Step 1: Standardize the data (subtract mean and divide by standard deviation)
    #standardized_data = [
        #[
            #(matrix[i][j] - sum(matrix[i]) / num_features) / (variances[i][j]**0.5)
            #for j in range(num_features)
        #]
        #for i in range(num_samples)
    #] Done in calculate_statistics_matrix_population 

    # Step 2: Calculate the covariance matrix
    covariance_matrix = [
        [
            sum(matrix[i][k] * matrix[j][k] for k in range(num_features)) / (num_samples - 1)
            for j in range(num_samples)
        ]
        for i in range(num_samples)
    ]

    return covariance_matrix


#eigenvalues and eigenvectors
def dot_product(v1, v2):
    return sum(x * y for x, y in zip(v1, v2))

def scalar_multiply(scalar, vector):
    return [[scalar * float(x) for x in subvector] for subvector in vector]

def vector_add(v1, v2):
    return [x + y for x, y in zip(v1, v2)]

def normalize_vector(v):
    norm = sum(x**2 for x in v)**0.5
    return [x / norm for x in v]

def matrix_vector_multiply(matrix, vector):
    return [dot_product(row, vector) for row in matrix]

def outer_product(vector1, vector2):
    return [[x * y for y in vector2] for x in vector1]


def subtract_matrices(matrix1, matrix2):
    return [[x - y for x, y in zip(row1, row2)] for row1, row2 in zip(matrix1, matrix2)]

def power_iteration(matrix, num_iterations=1000):
    num_rows, num_cols = len(matrix), len(matrix[0])

    # Initialize a random vector
    eigen_vector = [1.0] * num_cols

    for _ in range(num_iterations):
        # Compute the matrix-vector product
        matrix_times_vector = matrix_vector_multiply(matrix, eigen_vector)

        # Compute the new eigenvalue
        eigenvalue = dot_product(matrix_times_vector, eigen_vector)

        # Normalize the eigenvector
        eigen_vector = normalize_vector(matrix_times_vector)

    return eigenvalue, eigen_vector

def calculate_eigenvalues_and_eigenvectors(matrix, num_iterations=1000):
    num_rows, num_cols = len(matrix), len(matrix[0])

    eigenvalues = []
    eigenvectors = []

    for _ in range(min(num_rows, num_cols)):
        # Use power iteration to find the dominant eigenvalue and eigenvector
        eigenvalue, eigenvector = power_iteration(matrix, num_iterations)

        # Append the results to the lists
        eigenvalues.append(eigenvalue)
        eigenvectors.append(eigenvector)

        # Deflate the matrix by subtracting the outer product of the eigenvector
        outer = outer_product(eigenvector, eigenvector)
        matrix = subtract_matrices(matrix, scalar_multiply(eigenvalue, outer))

    return eigenvalues, eigenvectors


#project
def calculate_projection(data, eigenvectors):
    return [[sum(data[i][j] * eigenvectors[j][k] for j in range(len(eigenvectors))) for k in range(num_components)] for i in range(len(data))]


# Assuming 'data' is your matrix
data = croppedMatrix



means_matrix, variances_matrix, z_scores = calculate_statistics_matrix_population(data)
matrix = z_scores
#print(matrix)
Cmatrix = calculate_covariance_matrix(matrix, variances_matrix)
#print(Cmatrix)
# Display the results
#for i in range(len(data)):
    #for j in range(len(data[i])):
        #print(f"Element ({i + 1}, {j + 1}) - Mean: {means_matrix[i]}, Population Variance: {variances_matrix[i][j]}, Z-Score: {z_scores[i][j]}")

eigenvalues, eigenvectors = calculate_eigenvalues_and_eigenvectors(Cmatrix)
# Display the results
#for i, (eigenvalue, eigenvector) in enumerate(zip(eigenvalues, eigenvectors)):
    #print(f"Eigenvalue {i + 1}: {eigenvalue}")
    #print(f"Eigenvector {i + 1}: {eigenvector}")
    #print()
    
# Sort eigenvalues and eigenvectors in descending order
sorted_indices = sorted(range(len(eigenvalues)), key=lambda k: eigenvalues[k], reverse=True)
sorted_eigenvalues = [eigenvalues[i] for i in sorted_indices]
sorted_eigenvectors = [eigenvectors[i] for i in sorted_indices]

# Choose the number of components to retain (e.g., top 2 components)
num_components = 2
selected_eigenvalues = sorted_eigenvalues[:num_components]
selected_eigenvectors = sorted_eigenvectors[:num_components]

projected_data = calculate_projection(Cmatrix, selected_eigenvectors)

# Display the results
#print("Sorted Eigenvalues:", sorted_eigenvalues)
#print("Selected Eigenvalues:", selected_eigenvalues)
#print("Selected Eigenvectors:")
#for i, eigenvector in enumerate(selected_eigenvectors):
    #rint(f"Eigenvector {i + 1}: {eigenvector}")

print("\nProjected Data:")
for row in projected_data:
    print(row)


Projected Data:
[4.653901769956962, 2.9524303603604967]
[4.200850959527198, 1.9100464396084567]
[4.200850959527198, 1.9100464396084567]
[4.200850959527198, 1.9100464396084567]
[1.702317526795575, 1.3316047603708454]
[1.702317526795575, 1.3316047603708454]
[4.200850959527198, 1.9100464396084567]
[4.653901769956962, 2.9524303603604958]
[4.200850959527198, 1.9100464396084567]
[4.200850959527198, 1.9100464396084567]


In [37]:
#SKLEARNIMPLEMENTATION SVD

from sklearn.decomposition import TruncatedSVD
import numpy as np

def perform_svd_sklearn(matrix, num_components=None):
    # Convert the input matrix to a NumPy array
    matrix_np = np.array(matrix)

    # If num_components is None, set it to the minimum of the number of rows and columns
    if num_components is None:
        num_components = min(matrix_np.shape)

    # Perform SVD using scikit-learn
    svd = TruncatedSVD(n_components=num_components)
    svd.fit(matrix_np)

    # U, Sigma, and V matrices
    U_matrix = svd.transform(matrix_np)
    S_matrix = np.diag(svd.singular_values_)
    V_matrix = svd.components_

    return U_matrix, S_matrix, V_matrix


given_data = croppedMatrix
U_sklearn, S_sklearn, V_sklearn = perform_svd_sklearn(given_data)

print("U_matrix (sklearn):")
print(U_sklearn)

print("\nS_matrix (sklearn):")
print(S_sklearn)

print("\nV_matrix (sklearn):")
print(V_sklearn)



U_matrix (sklearn):
[[ 5.47631357e-01 -3.17557020e-02  9.65604036e-02  2.41393439e-02
  -1.26773640e-03  4.82918930e-04 -2.50355126e-08  6.20575480e-06
   1.52675072e-06 -1.15782498e-18]
 [ 5.09357854e-01  1.78518328e-02 -1.69182336e-02 -7.53760473e-03
   1.05339992e-03 -6.49750551e-04 -2.02884975e-05  3.01692367e-05
   2.74306196e-06  4.62687087e-19]
 [ 7.48891685e-01 -7.40741941e-03 -2.53293767e-02  3.05178200e-03
  -6.96940509e-03 -1.26130630e-03  2.84185762e-05 -6.62840991e-06
  -7.08720238e-06 -5.01813310e-19]
 [ 5.88204403e-01  1.58239201e-02 -3.46912399e-02 -7.19982331e-03
   4.15209971e-04  1.32804138e-03  7.92283692e-05  2.88211720e-06
   9.96608847e-07 -1.98506928e-18]
 [ 2.37491672e-02  1.26275036e-01  4.93689507e-02 -2.34397611e-03
   1.23197874e-04 -8.25500906e-05  3.05149034e-06 -1.68827741e-06
  -1.34686343e-06  1.32775836e-17]
 [ 2.37491672e-02  1.26275036e-01  4.93689507e-02 -2.34397611e-03
   1.23197874e-04 -8.25500906e-05  3.05149034e-06 -1.68827741e-06
  -1.34686343

In [45]:
#SKLEARNIMPLEMENTATION PCA

from sklearn.decomposition import PCA
import numpy as np

def perform_pca(matrix, num_components):
    # Create PCA object
    pca = PCA(n_components=num_components)

    # Fit the model and transform the data
    reduced_matrix = pca.fit_transform(matrix)

    # Get the principal components (eigenvectors) and explained variance
    principal_components = pca.components_
    explained_variance = pca.explained_variance_ratio_

    return reduced_matrix, principal_components, explained_variance


data = croppedMatrix

num_components = 2
reduced_data, principal_components, explained_variance = perform_pca(data, num_components)

print("Reduced Data:")
print(reduced_data)

print("\nPrincipal Components:")
print(principal_components)

print("\nExplained Variance:")
print(explained_variance)


Reduced Data:
[[-1.12001008e-01  8.18411675e-02  6.11165845e-02  5.22652236e-04]
 [-8.20795375e-02 -2.23485858e-02 -1.67371517e-02  5.20905073e-04]
 [-3.22068366e-01 -1.45292423e-02  6.70339900e-03 -6.99420082e-03]
 [-1.62142250e-01 -3.35046454e-02 -1.82730262e-02 -1.35366478e-04]
 [ 4.17849835e-01 -5.69660693e-02  1.26733529e-02 -1.80450099e-04]
 [ 4.17849835e-01 -5.69660693e-02  1.26733529e-02 -1.80450099e-04]
 [-1.92135377e-01 -2.65675939e-02 -1.07873345e-02 -1.12216632e-03]
 [ 2.79150367e-01  1.38927836e-01 -3.43861262e-02 -2.78461625e-03]
 [-1.11983894e-01 -1.80490221e-02 -5.71239857e-03 -4.37561788e-03]
 [-1.32439604e-01  8.16222444e-03 -7.27065224e-03  1.47293106e-02]]

Principal Components:
[[ 1.03811898e-03 -9.99487449e-01 -2.92752685e-03 -1.74021832e-02
   1.65141579e-05 -1.03016398e-04  2.66895357e-02 -1.49467484e-05
  -2.78360952e-06 -1.14615046e-04]
 [ 8.06279578e-01  5.19751710e-03  1.57960272e-02  4.06060813e-01
   7.68618131e-04  3.10609033e-03  4.29799646e-01 -1.052738

Unfortunately, my SVD from scratch implementation is extremely slow. My PCA is faster than it, but it is still a lot slower than the implementations with sklearn and numpy. The data downloaded for this exam was converted into a matrix with 2250 rows and 83 columns. The matrix was then cropped as my scratch code could not handle the size within the time frame. One of the attempts cropped the matrix into a 40 by 25 dimension matrix. However, after 110 minutes and 26.3 seconds my SVD scratch code was still computing the Sigma matrix. On the other hand, my PCA scratch code finished in 9.6 seconds while the sklearn SVD and PCA finished at 0.0 seconds and 0.4 seconds respectively. At this stage, I could see the gap between my non-package and package results, especially in the PCA computations. To be able to inspect it further, I cropped the matrix into a 10 by 10 matrix - keeping the first 10 rows and columns. I will paste the results of that computation below. 

As a whole, the creation of the U, Sigma, and V matrices are different for both SVD implementations. The sklearn SVD implementation uses TruncatedSVD's fit to set the dimensionality of the matrix equal to the number of rows if it is smaller than the number of columns (and vice-versa). Using num_components, it computes the SVD of the data and then decides which to keep according to the dimensionality determined at the start. Therefore it may operate to find U, Sigma, and V and output a different matrix than the scratch SVD, which operates on the given matrix and keeps it as is. As the test matrix was a square in this case, I do not think it affected the package and non-package SVD implementation. In my SVD scratch implementation, I first created the transpose function and then created separate matrix multiplication functions to compute for the matrices U and V. function multiplyU multiplies the original matrix to the transposed matrix and then calls the function eigen which calculates for the eigenvalues and eigenvectors of the product while normalizing them implicitly. When I tested the code on a small matrix (2 x 3) these functions worked well. However, I learned that power iteration (for finding eigenvalues and vectors) only works on symmetric matrices and the QR algorithm was more suited to the task. I could not get the QR algorithm to work, so I kept the power iteration method. This step affects the V matrix, which follows a similar process, as well as the Sigma matrix. Because the Sigma matrix is a diagonal of the eigenvalues' squaroots, it also differs from the sklearn function's Sigma matrix. However, the first three singular values are the same in this case. This could be because the two programs use different processes to compute eigenvalues and eigenvectors - sklearn uses random factorization. To summarize, I believe differences between the sklearn and non-sklearn come from the lack of truncation in my code and the process of computing eigenvalues and eigenvectors through power iteration instead of using randomization factorization.

To compute for the PCA value, I first created the function calculate_statistics_matrix_population to standardize the data in the matrix. This function first calculates for the row mean. Then, it calculates the variance of each index using their row's mean. Afterwards, the variance and row mean is used to calculate for the Z-score. The Z-score of each index is put into a new matrix that is passed to the calculate_covariance_matrix. The covariance matrix is then calculated by iterating through each variable and applying the covariance formula. This process is different to how sklearn performs the standardization calculation. In the sklearn implementation, num_components is used to truncate the matrix using the fit_transform method. The fit_transform method also standardizes the data by column through subtracting the mean and possibly dividing by the standard deviation. Additionally, I used the population variance model in the PCA sratch code while sample standard deviation is used in the sklearn library. With the next functions, the PCA scratch code computes for eigenvalues and eigenvectors and arranges them in descending order. PCA sklearn does this as well with the fit_transform method. In summary, the difference in results is caused by the standardization in the calculate_statistics_matrix_population. 





Result of cropping to 10 x 10

SVD: 17.2s

U
[0.3480254450741242, -0.1580042068278438, 0.5644379026803268, -0.6712302839050692, 0.07884897485149034, -0.2035660496031232, -0.15750768367079232, -0.00021703285682973423, -0.17157123102261795, 0.5637423165512199]
[0.3237016800833751, 0.08934927105961579, -0.09913037452766019, 0.20951771630920069, -0.06541725359507795, 0.273885585995372, 0.08925791956502722, -0.17157896386799176, -0.8352434248911778, -0.09932770177171513]
[0.47592760076351637, -0.03755572418356156, -0.14785811430957044, -0.08462153208801577, 0.43241632093032206, 0.5315965744299753, -0.037694186270989555, 0.2403211059626636, 0.18342477993565, -0.14786355285435973]
[0.37380929800916873, 0.0789135659772076, -0.20290501950416703, 0.20023466535311515, -0.025819314735428124, -0.5597803250053485, 0.07874616615112612, 0.6700089464070009, -0.07992213229342612, -0.20285930088993107]
[0.01509337133782324, 0.6345044973581255, 0.28622554052736265, 0.06485121053575073, -0.007675651779765533, 0.034796733092162656, 0.6347541246502263, 0.025801889769173508, 0.04685421486076462, 0.28630393672613774]
[0.01509337133782324, 0.6345044973581255, 0.28622554052736265, 0.06485121053575073, -0.007675651779765533, 0.034796733092162656, 0.6347541246502263, 0.025801889769173508, 0.04685421486076462, 0.28630393672613774]
[0.3931745529416182, 0.05046265385743363, -0.16903891901865967, 0.11613894462548695, 0.04434249592294062, -0.46494432923907203, 0.05032316546283594, -0.650464582487553, 0.10855507543888686, -0.16878289245615685]
[0.09299101990455136, -0.37739499284249756, 0.6425845607723009, 0.6475069349275138, 0.04874348539501532, 0.021967656823929857, -0.37683429508504607, 0.029411186977135226, 0.06088406055796404, 0.6432047632443145]
[0.34348657838714125, 0.08221468781976572, -0.05663468868961935, 0.12061926679079393, 0.2376150899543891, 0.08962408444100513, 0.08216367885856225, -0.19449642726263267, 0.3837412431540555, -0.05629790875679649]
[0.35522222809533566, -0.04988953583696533, -0.0019677417035085672, -0.05082967483962205, -0.8607740492887626, 0.24594452627900754, -0.04989557533975751, 0.027656167102977337, 0.25643807616117376, -0.0019253113382916472]
Sigma
[1.5735406362361206, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0.19923607426168852, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0.17121648909707052, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0.035991535293459595, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0.01611866280677628, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0.0023724184646813683, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, (9.156991382437214e-21+0.0001495450180217295j), 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0.00011825110372116888, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 3.612148775276756e-05, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, (2.0171709225350144e-21+3.294290115222523e-05j)]
V
[0.06567209858578014, 0.982227937740972, 0.0026966942640839608, -0.10300913819729582, -0.036504112914811385, 0.039075753838402075, -0.11459998971927732, -0.03736070815482227, -0.03735390385539359, 0.03890424125838363]
[0.361976389057463, 0.13473259158455062, 0.012563610540518811, 0.6667219416213567, 0.09965068461076189, -0.10369281598292429, 0.5952104739769424, 0.10150072070003474, 0.10147970350088145, -0.1028870103253729]
[0.8981346833198016, -0.12733739321629498, 0.009600996984094196, -0.2509152932942955, -0.13526294509730125, 0.1474513842486803, -0.11688156944081046, -0.1391195133560955, -0.1390241326528515, 0.14733428178926497]
[0.2375231459375903, 0.0008406901733414552, 0.012114570565916777, 0.0337732877778511, 0.36006030102694964, -0.3826811489493479, -0.5008802800705653, 0.3690267098951952, 0.36878622562449836, -0.37834021815002156]
[-0.028916680322933107, -0.029571448984723178, 0.22529248534104268, 0.6743321247313754, -0.1638179472284508, 0.17389086135402876, -0.5907038630610767, -0.16735617978459613, -0.16715362502302036, 0.1756559307449385]
[-0.010189941990245002, 0.0036220579062477606, 0.9731760042851859, -0.16216317718581316, 0.06153969484156334, -0.006543812240655393, 0.13665317533024665, 0.042570623219977184, 0.039242821879870944, -0.02565758268380606]
[0.36282867844788436, 0.13461179020838365, 0.012551526582700893, 0.6664877732574104, 0.09952108720843858, -0.10355285712101052, 0.5950969014625201, 0.10136787913062668, 0.10134702490921906, -0.10274674586925225]
[-0.0027455583344963533, -0.00018861879113301307, -0.02711912910591477, 0.003866493489494766, -0.04655436046986187, 0.7439663186771972, -0.0037261847560777447, 0.5106527114014204, 0.41386043020054386, 0.10736994827193998]
[-0.000558945733702207, 0.00011544061792315002, 0.019755674548557003, -0.004412282244442037, -0.5630414561493653, -0.4825947192793062, 0.0045585815497140416, 0.35250786058395733, 0.23556554510106179, 0.5195217766798643]
[-0.0033701905453595044, -9.133377809177129e-05, -0.023947255253178115, 0.002295218265823883, 0.677555616112401, -0.05152323226841092, -0.0018388228765880484, -0.12450946629447457, 0.1367516660467599, 0.7095538255769392]

SVD (sklearn): 0.0s


U_matrix (sklearn):
[[ 5.47631357e-01 -3.17557020e-02  9.65604036e-02  2.41393439e-02
  -1.26773640e-03  4.82918930e-04 -2.50355126e-08  6.20575480e-06
   1.52675072e-06 -1.15782498e-18]
 [ 5.09357854e-01  1.78518328e-02 -1.69182336e-02 -7.53760473e-03
   1.05339992e-03 -6.49750551e-04 -2.02884975e-05  3.01692367e-05
   2.74306196e-06  4.62687087e-19]
 [ 7.48891685e-01 -7.40741941e-03 -2.53293767e-02  3.05178200e-03
  -6.96940509e-03 -1.26130630e-03  2.84185762e-05 -6.62840991e-06
  -7.08720238e-06 -5.01813310e-19]
 [ 5.88204403e-01  1.58239201e-02 -3.46912399e-02 -7.19982331e-03
   4.15209971e-04  1.32804138e-03  7.92283692e-05  2.88211720e-06
   9.96608847e-07 -1.98506928e-18]
 [ 2.37491672e-02  1.26275036e-01  4.93689507e-02 -2.34397611e-03
   1.23197874e-04 -8.25500906e-05  3.05149034e-06 -1.68827741e-06
  -1.34686343e-06  1.32775836e-17]
 [ 2.37491672e-02  1.26275036e-01  4.93689507e-02 -2.34397611e-03
   1.23197874e-04 -8.25500906e-05  3.05149034e-06 -1.68827741e-06
  -1.34686343e-06  1.32775836e-17]
 [ 6.18676381e-01  1.01389466e-02 -2.89082037e-02 -4.17408935e-03
  -7.15274408e-04  1.10302844e-03 -7.69190684e-05 -3.92192950e-06
  -6.38734166e-06 -1.15233461e-18]
 [ 1.46324335e-01 -7.55057000e-02  1.09801493e-01 -2.33267953e-02
  -7.89525747e-04 -5.21318385e-05  3.47874155e-06 -2.18978490e-06
  -1.57422566e-06 -1.23989005e-17]
 [ 5.40490130e-01  1.64095832e-02 -9.64545830e-03 -4.33875611e-03
  -3.83063891e-03 -2.12701027e-04 -2.29996893e-05 -1.38616208e-05
   1.29528530e-05 -4.18810070e-18]
 [ 5.58956645e-01 -9.93704055e-03 -3.59680185e-04  1.82745526e-03
   1.38748094e-02 -5.83211020e-04  3.27047807e-06 -9.26303139e-06
  -4.77371566e-07 -7.25849385e-18]]

S_matrix (sklearn):
[[1.57354064e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 1.99236233e-01 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 1.71216239e-01 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 3.59915196e-02
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  1.61186621e-02 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 2.37241844e-03 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 1.18251103e-04 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 3.61214920e-05
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  1.66128177e-05 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 8.47260001e-19]]

V_matrix (sklearn):
[[ 6.56719450e-02  9.82227896e-01  2.69668966e-03 -1.03009357e-01
  -3.65041423e-02  3.90757842e-02 -1.14600188e-01 -3.73607380e-02
  -3.73539337e-02  3.89042713e-02]
 [-3.59569632e-01 -1.35093734e-01 -1.25378716e-02 -6.67387824e-01
  -1.00011646e-01  1.04086349e-01 -5.95517492e-01 -1.01871984e-01
  -1.01850712e-01  1.03280238e-01]
 [ 8.99444247e-01 -1.26845401e-01  9.64650464e-03 -2.48496501e-01
  -1.34902404e-01  1.47076228e-01 -1.14720778e-01 -1.38752283e-01
  -1.38656978e-01  1.46962033e-01]
 [ 2.37599859e-01  8.37470210e-04  1.20640385e-02  3.36001845e-02
   3.60086935e-01 -3.82709118e-01 -5.00754846e-01  3.69053846e-01
   3.68813323e-01 -3.78368600e-01]
 [-2.86467291e-02 -2.95705664e-02  2.25305317e-01  6.74370770e-01
  -1.63409517e-01  1.73456712e-01 -5.91272711e-01 -1.66937557e-01
  -1.66735272e-01  1.75226728e-01]
 [-1.01912869e-02  3.62065806e-03  9.73186777e-01 -1.62130875e-01
   6.15318756e-02 -6.53552735e-03  1.36624999e-01  4.25626227e-02
   3.92348332e-02 -2.56491992e-02]
 [ 2.74564543e-03  1.88586215e-04  2.71106633e-02 -3.86509492e-03
   4.65475011e-02 -7.43971682e-01  3.72501045e-03 -5.10649130e-01
  -4.13858130e-01 -1.07363883e-01]
 [ 5.65669203e-04 -1.15409320e-04 -1.97531411e-02  4.41286989e-03
   5.63086897e-01  4.82492390e-01 -4.57202801e-03 -3.52564639e-01
  -2.35599144e-01 -5.19513775e-01]
 [-3.27953131e-03 -9.10141198e-05 -2.39384374e-02  2.30744150e-03
   6.77575612e-01 -5.17698253e-02 -2.02952657e-03 -1.24294570e-01
   1.36942011e-01  7.09517910e-01]
 [-3.44050726e-04 -3.29577772e-05 -6.61224286e-03  9.94622259e-04
   1.78809349e-01 -2.09427681e-02 -8.56000561e-04  6.34492012e-01
  -7.47063214e-01  8.28220710e-02]]



PCA: 0.7s

Projected Data:
[130.83699946568746, 91.29006459083367]
[152.8408084231713, 112.0181580607778]
[70.72839380712809, 52.33267742183428]
[111.0178833730709, 81.1761837358259]
[2915.492647487677, 2068.248017702202]
[2915.492647487677, 2068.248017702202]
[98.70886644476894, 72.39649378025625]
[865.7893075818553, 621.2710019452494]
[134.41366271652893, 99.12593179114216]
[128.87424351386557, 98.66265623175917]

PCA (sklearn) 0.0s

Reduced Data:
[[-0.11200101  0.08184117]
 [-0.08207954 -0.02234859]
 [-0.32206837 -0.01452924]
 [-0.16214225 -0.03350465]
 [ 0.41784983 -0.05696607]
 [ 0.41784983 -0.05696607]
 [-0.19213538 -0.02656759]
 [ 0.27915037  0.13892784]
 [-0.11198389 -0.01804902]
 [-0.1324396   0.00816222]]

Principal Components:
[[ 1.03811898e-03 -9.99487449e-01 -2.92752685e-03 -1.74021832e-02
   1.65141579e-05 -1.03016398e-04  2.66895357e-02 -1.49467484e-05
  -2.78360952e-06 -1.14615046e-04]
 [ 8.06279578e-01  5.19751710e-03  1.57960272e-02  4.06060813e-01
   7.68618131e-04  3.10609033e-03  4.29799646e-01 -1.05273878e-05
   2.98612599e-05  3.65425503e-03]]

Explained Variance:
[0.93896929 0.0516908 ]